In [2]:
import numpy as np
from numpy.random import permutation
import time
import pandas as pd
import matplotlib.pyplot as plt
fw, fh = plt.rcParams["figure.figsize"]
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import sklearn.model_selection


# Etape 0 : Récupérer les données

Pierre a généré le dataset et l'a annoté de labels qui correspondent au pixel où il regarde. On a donc en x ??? et en y un tuple d'integer qui correspondent respectivement à la coordonnée x et la coordonnée y. La matrice est sparse.

Accès au Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = "/content/drive/MyDrive/Travail/eyes_detection/"

Lecture des fichiers textes

In [6]:
array_files = []
for i in range(1,65):
  file_path = path + "pixels_faces_"+str(i)+".txt"
  file_tmp = open(file_path, "r")
  array_files.append(file_tmp.read())

Vérification qu'on a lu tous les fichiers

In [7]:
print(np.shape(array_files))
n = np.shape(array_files)[0]
print(n)

(64,)
64


Conversion en liste de chaque fichier texte

In [8]:
list_files = []
for i in range (1,np.shape(array_files)[0]):
  file_array = [int(position) if position.isdigit() else position for position in array_files[i].split(',')]
  list_files.append(file_array)

Vérification

In [9]:
print(np.shape(list_files[50]))

(19901,)


Lecture des labels y

Le label doit être transformé en tuple de string pour qu'on puisse ensuite créer les sets d'apprentissages et de tests. On doit avoir à la fin y qui est une liste de tuple mais de dimensions 1 pour le training et le test set.

In [22]:
from ast import literal_eval
file_path = path + "points_increment_1.txt"
file_tmp = open(file_path, "r")
tmp = (literal_eval(file_tmp.read()))

In [23]:
array_files_labels = []
for i in range(1,n+1):
  file_path = path + "points_increment_"+str(i)+".txt"
  file_tmp = open(file_path, "r")
  array_files_labels.append(list(literal_eval(file_tmp.read())))

In [24]:
print(type(array_files_labels[0]))
print(array_files_labels[0])
print(np.shape(array_files_labels))

<class 'list'>
[536, 44]
(88, 2)


Vérification qu'on a bien tout les labels

In [25]:
print(np.shape(array_files_labels))

if np.shape(array_files_labels)[0] == n:
  print("Ok")
else:
  print("Le nombre de labels n'est pas le bon.")

(88, 2)
Ok


On a le bon nombre d'image et de labels, les données d'entrée sont correctes.

# Etape 1 : Réduction des données

C'est pas mal de réduire les données parce qu'une image, ça prend de la place. On gagnerait en temps de calcul mais on perdrait peut être en précision. Il faut voir combien de dimensions on garde. Après, si l'image est en noir et blanc, nous aurons déjà 2 dimensions.

Parmi les réductions de dimensions, t-SNE et Umap sont les meilleures (voir APPC-CM11)

# Etape 2 : Le modèle d'apprentissage

On a un problème de régression car il doit prédire parmi un nombre suffisament grand de classes pour qu'on considère qu'il est infini.

On va utiliser du deep learning pour extraire les caractéristiques et au lieu d'avoir la couche de décision, on remplace cette couche par notre modèle en machine learning de régression.

## Extraction des caractéristiques avec le réseau de neurones

Import des biblio

In [12]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

Création du modèle VGG16 déjà pré entrainé sur un problème de classification (à voir pour trouver un modèle de régression ?)

In [13]:
model = VGG16(weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


Définition du nombre d'images

In [14]:
n = 88

Extraction des features (1min)

https://keras.io/api/applications/#extract-features-with-vgg16

In [15]:
array_features = []
for i in range(1,n+1):
  img_path = path + "eyes_"+ str(i)+".png"
  #vérifiez pour la target-size à quoi ça correspond
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  #Extraction des features
  features = model.predict(x)
  #On crée un tableau pour y stocker chaque feature d'image
  array_features.append(features)

1/1 [==============================] - 0s 469ms/step


https://stackoverflow.com/questions/47555829/preprocess-input-method-in-keras

Vérification qu'on a bien toutes les images qui ont été traitées.

In [16]:
print(np.shape(array_features))

if np.shape(array_features)[0] == n:
  print("Ok")
else:
  print("Le nombre d'images n'est pas le bon.")

(88, 1, 7, 7, 512)
Ok


Marice sparse ou pas ?

In [18]:
tmp = array_features[0]
print(np.count_nonzero(tmp))
print(np.unique(tmp, return_counts=True))

2092
(array([0.0000000e+00, 9.5178410e-03, 1.6311496e-02, ..., 1.6911426e+02,
       1.7268654e+02, 1.7467073e+02], dtype=float32), array([22996,     1,     1, ...,     1,     1,     1]))


On remarque qu'il y a 10 fois plus de valeurs à 0 que pas à 0.

Une fois les features extraites, on peut récupérer les labels connus pour pouvoir ensuite les attribuer à leurs features.

## Centrer les données

 For VGG16, call keras.applications.vgg16.preprocess_input on your inputs before passing them to the model. vgg16.preprocess_input will convert the input images from RGB to BGR, then will zero-center each color channel with respect to the ImageNet dataset, without scaling.

## Partitionner les données

Comme on a extrait les caractéristiques, on peut maintenant entrainé les modèles de régressions linéaires dessus.

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(array_features_final, array_files_labels, test_size=0.3, random_state=2020)

## Modèles de régression linéaire

### Moindre carrés

Linéaire simple

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
linear_regression = LinearRegression()
linear_regression.fit(X_train,y_train)

LinearRegression()

In [32]:
# Prédictions sur les données de test
predictions = linear_regression.predict(X_test)

# Calculer l'erreur quadratique moyenne
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error :", mse)

Mean Squared Error : 16911.048102918285


### Ridge Regression

Linéaire simple

In [33]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(X_train,y_train)

# Prédictions sur les données de test
predictions_ridge = ridge.predict(X_test)

# Calculer l'erreur quadratique moyenne
mse_ridge = mean_squared_error(y_test, predictions_ridge)
print("Mean Squared Error :", mse_ridge)

Mean Squared Error : 16977.816894983032


### Lasso Regression

Linéaire simple

In [34]:
from sklearn.linear_model import MultiTaskLasso
lambda_lasso = 10e-3
multitask_lasso = MultiTaskLasso(alpha=lambda_lasso*n, fit_intercept=False)
beta_lasso = multitask_lasso.fit(X_train,y_train).coef_
print(np.shape(beta_lasso))
print(np.shape(X_train))

(2, 25088)
(61, 25088)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:2418: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25864.255859375, tolerance: 7817.95654296875
  ) = cd_fast.enet_coordinate_descent_multi_task(


In [36]:
# Prédictions sur les données de test
predictions_lasso = multitask_lasso.predict(X_test)

# Calculer l'erreur quadratique moyenne
mse_lasso = mean_squared_error(y_test, predictions_lasso)
print("Mean Squared Error :", mse_lasso)

Mean Squared Error : 185016.43783221586


### Multitask Elastic Net

C'est Lasso + Ridge pour les pénalités. Régression multiple.

In [40]:
from sklearn.linear_model import MultiTaskElasticNetCV

MTelasticnetCV = MultiTaskElasticNetCV(alphas=[lambda_lasso*n])
MTelasticnetCV.fit(X_train,y_train)

# Prédictions sur les données de test
predictions_MTElasticnetCV = MTelasticnetCV.predict(X_test)

# Calculer l'erreur quadratique moyenne
mse_MTENCV = mean_squared_error(y_test, predictions_MTElasticnetCV)
print("Mean Squared Error :", mse_MTENCV)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7910.98486328125, tolerance: 1814.4599609375
  model = cd_fast.enet_coordinate_descent_multi_task(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7986.6142578125, tolerance: 1863.449951171875
  model = cd_fast.enet_coordinate_descent_multi_task(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8134.27294921875, tolerance: 1862.1787109375
  model = cd_fast.enet_coordinate_descent_multi_task(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:609: Conv

Mean Squared Error : 85200.2744435561


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:2418: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10228.0849609375, tolerance: 2353.7734375
  ) = cd_fast.enet_coordinate_descent_multi_task(


# Etape 3 : Améliorer le modèle avec du cross-validation/GridSearchCV

### LassoMultiTaskCV

Plusieurs alphas, régression multiple

In [53]:
from sklearn.linear_model import MultiTaskLassoCV
alphas = np.logspace(-6, 6, 13)
MTLassoCV = MultiTaskLassoCV(alphas=alphas,max_iter=3000)
MTLassoCV.fit(X_train,y_train)

# Prédictions sur les données de test
predictions_MTLassoCV = MTLassoCV.predict(X_test)

# Calculer l'erreur quadratique moyenne
mse_MTLCV = mean_squared_error(y_test, predictions_MTLassoCV)
print("Mean Squared Error :", mse_MTLCV)

Mean Squared Error : 18156.472349386004


Un seul alpha pour voir s'il y a vraiment une différence.

In [54]:
from sklearn.linear_model import MultiTaskLassoCV
MTLassoCV = MultiTaskLassoCV(max_iter=3000)
MTLassoCV.fit(X_train,y_train)

# Prédictions sur les données de test
predictions_MTLassoCV = MTLassoCV.predict(X_test)

# Calculer l'erreur quadratique moyenne
mse_MTLCV = mean_squared_error(y_test, predictions_MTLassoCV)
print("Mean Squared Error :", mse_MTLCV)

Mean Squared Error : 18183.92899783985


Légère amélioration avec plusieurs alphas pour la cross validation mais c'est pas fou comme résultat.

### Linear Regression CV

Régression simple

In [43]:
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline

pipe_linear = Pipeline([("linear", LinearRegression())])
param_grids = {}
search_linear = GridSearchCV(pipe_linear,param_grids,n_jobs=2,cv=20)
search_linear.fit(X_train, y_train)
predictions_linear = search_linear.predict(X_test)
# Calculer l'erreur quadratique moyenne
mse_linear_cv = mean_squared_error(y_test, predictions_linear)
print("Mean Squared Error :", mse_linear_cv)

Mean Squared Error : 16911.048102918285


### Ridge Regression CV

Régression simple

In [47]:
from sklearn.linear_model import RidgeCV

ridgeCV = RidgeCV(cv=20)
ridgeCV.fit(X_train,y_train)

# Prédictions sur les données de test
predictions_ridgeCV = ridgeCV.predict(X_test)

# Calculer l'erreur quadratique moyenne
mse_ridgeCV = mean_squared_error(y_test, predictions_ridgeCV)
print("Mean Squared Error :", mse_ridgeCV)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=8.13225e-09): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dis

Mean Squared Error : 16977.80848088926


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=4.16342e-09): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)


# Etape 4 : Calculer la précision/f-measure pour comparer

Faire un tableau qui compare toutes les mesures du mean square error.

# Etape 5 : Comparer

# Conclusion

De ce que j'ai compris, on fait de la régressions multiples car Y a plusieurs dimensions. Entre Lasso et ElasticNet, LassoCV donne de meilleurs résultats sur la mean square error parce que Lasso élimine normalement les variables les moins impoortantes. Or il y a beaucoup de valeurs à 0 qui ne donnent aucune information donc Lasso peut les ignorer.